In [6]:
import stanfordnlp
import json

In [7]:

class ReturnWord():
    def __init__(self):
        self.text = ""
        self.upos = ""
        self.xpos = ""
        self.case = ""
        self.gender = ""
        self.number = ""
        self.governor = ""
        self.relation = ""
        self.notes = []
        
    def set_vars(self, text, upos, xpos, governor, relation):
        self.text = text
        self.upos = upos
        self.xpos = xpos
        self.governor = governor
        self.relation = relation

nlp_de = stanfordnlp.Pipeline(lang='de')
#nlp_en = stanfordnlp.Pipeline(lang='en')

sentence = "Ich esse der Hut"

prep_akk = ['bis', 'durch', 'für', 'gegen', 'ohne', 'um']
prep_dat = ['aus', 'ausser', 'bei', 'nach', 'mit', 'seit', 'von', 'zu']
prep_acc_dat = ['an', 'auf', 'hinter', 'in', 'neben', 'über', 'unter', 'von', 'zwischen']

nominative = { 'Masc': 'der', 'Fem': 'die', 'Neut': 'das' }
accusative = { 'Masc': 'den', 'Fem': 'die', 'Neut': 'das' }
dative = { 'Masc': 'dem', 'Fem': 'der', 'Neut': 'den' }
genitive = { 'Masc': 'des', 'Fem': 'der', 'Neut': 'des' }

doc_de = nlp_de(sentence)

sentences_ret = []

for sentence in doc_de.sentences:
    words_ret = [ReturnWord() for _ in range(len(sentence.words))]
    for word in sentence.words:
        print("[%d] Word: %s" % (int(word.index), word.text))
        words_ret[int(word.index) - 1].set_vars(word.text, word.upos, word.xpos, word.governor, word.dependency_relation)
        print("%s, %s, %s, %s" % (words_ret[int(word.index) - 1].text, words_ret[int(word.index) - 1].upos, words_ret[int(word.index) - 1].xpos, words_ret[int(word.index) - 1].relation))
        if(word.upos == 'ADP'):
            print("%s is ADP" % word.text)
            if(word.lemma in prep_akk):
                print("%s is accusative" % word.text)
                words_ret[int(word.index) - 1].notes.append(word.text + " takes the accusative case.")
                words_ret[word.governor - 1].notes.append("Because " + word.text + " is accusative, so is " + sentence.words[word.governor - 1].text)
                words_ret[word.governor - 1].case = "Acc"
            elif(word.lemma in prep_dat):
                print("%s is dative" % word.text)
                words_ret[int(word.index) - 1].notes.append(word.text + " takes the dative case.")
                words_ret[word.governor - 1].notes.append("Because " + word.text + " is dative, so is " + sentence.words[word.governor - 1].text)
                words_ret[word.governor - 1].case = "Dat"
            elif(word.lemma in prep_acc_dat):
                print("%s is akk/dat" % word.text)
                words_ret[int(word.index) - 1].notes.append(word.text + " takes Dative if it answers the question 'where?'")
                words_ret[int(word.index) - 1].notes.append(word.text + " takes the Accusative if it answers the question 'where to?'")
                words_ret[word.governor - 1].notes.append(sentence.words[word.governor - 1].text + " takes either the Accusative or Dative case.")
                words_ret[word.governor - 1].case = "Acc|Dat"
        if(word.upos == 'NOUN'):
            words_ret[int(word.index) - 1].gender = (nlp_de(word.text).sentences[0].words[0].feats.split('|')[1].split('=')[1])
            print("%s, %s" % (words_ret[int(word.index) - 1].gender, word.text))
        
        feats = word.feats.split('|')
        for feat in feats:
            pair = feat.split('=')
            #print("%s: %s" % (word.text, pair))
            if(pair[0] == "Gender" and words_ret[int(word.index) - 1].gender == ""): words_ret[int(word.index) - 1].gender = pair[1]
            elif(pair[0] == "Number"): words_ret[int(word.index) - 1].number = pair[1]
            elif(pair[0] == "Case" and words_ret[int(word.index) - 1].case == ""): words_ret[int(word.index) - 1].case = pair[1]
            #if(len(pair) is 2): print("%s gender: %s (pair[1] is %s)" % (word.text, words_ret[int(word.index) - 1].gender, pair[1]))
        if(word.dependency_relation == 'obj' or word.dependency_relation == 'dobj'):
            words_ret[int(word.index) - 1].notes.append(word.text + " is probably a direct object, which takes the accusative case.")
            words_ret[int(word.index) - 1].case = "Acc"
        if(word.dependency_relation == 'iobj'):
            words_ret[int(word.index) - 1].notes.append(word.text + " is probably an indirect object, which takes the dative case.")
            words_ret[int(word.index) - 1].case = "Dat"
            
    for word in sentence.words:
        #print("%s == 'ART' ? %s" % (word.xpos, (word.xpos == 'ART')))
        if(word.xpos == 'ART'):
            print(words_ret[word.governor - 1].case)
            if('Acc' in words_ret[word.governor - 1].case):
                #print("%s is Acc %s" % (str(words_ret[word.governor - 1].text), str(words_ret[word.governor - 1].case)))
                nom = nominative[words_ret[word.governor - 1].gender]
                article = accusative[words_ret[word.governor - 1].gender]
                words_ret[int(word.index) - 1].notes.append("%s %s takes the accusative case, so the article should be %s" % (nom, words_ret[word.governor - 1].text, article))
            if('Dat' in words_ret[word.governor - 1].case):
                #print("%s is Dat %s" % (str(words_ret[word.governor - 1].text), str(words_ret[word.governor - 1].case)))
                nom = nominative[words_ret[word.governor - 1].gender]
                article = dative[words_ret[word.governor - 1].gender]
                words_ret[int(word.index) - 1].notes.append("%s %s takes the dative case, so the article should be %s" % (nom, words_ret[word.governor - 1].text, article))
            if('Gen' in words_ret[word.governor - 1].case):
                #print("%s is Gen %s" % (str(words_ret[word.governor - 1].text), str(words_ret[word.governor - 1].case)))
                nom = nominative[words_ret[word.governor - 1].gender]
                article = genitive[words_ret[word.governor - 1].gender]
                words_ret[int(word.index) - 1].notes.append("%s %s takes the genitive case, so the article should be %s" % (nom, words_ret[word.governor - 1].text, article))
print("this is output",json.dumps(words_ret[0]))

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/sanskritisharma/stanfordnlp_resources/de_gsd_models/de_gsd_tokenizer.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: mwt
With settings: 
{'model_path': '/Users/sanskritisharma/stanfordnlp_resources/de_gsd_models/de_gsd_mwt_expander.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
---
Loading: pos
With settings: 
{'model_path': '/Users/sanskritisharma/stanfordnlp_resources/de_gsd_models/de_gsd_tagger.pt', 'pretrain_path': '/Users/sanskritisharma/stanfordnlp_resources/de_gsd_models/de_gsd.pretrain.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/sanskritisharma/stanfordnlp_resources/de_gsd_models/de_gsd_lemmatizer.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an a

../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


TypeError: Object of type ReturnWord is not JSON serializable

NameError: name 'version' is not defined